In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import colorsys
import json, os

In [ ]:
# Resources
do_save_plots = True
data_folder = os.path.join("..", "..", "results", "for_plots")
panels_folder = "panels/"
params_folder = os.path.join("..", "..", "results", "common_params")

# Aesthetic parameters -- main definition
They are defined here for figure 1, and loaded back in other figures. 

In [ ]:
# Colors
back_choice = "greys"  # or "blues", or "colorblind"
if back_choice == "greys":
    back_palette = [a + (1,) for a in sns.dark_palette((0.85, 0.85, 0.85, 1.0), n_colors=3)]
    back_palette[0] = mpl.colors.to_rgba("k")
    back_color = back_palette[0]
    back_color_samples = back_palette[1]
elif back_choice == "blues":
    back_color = np.asarray(mpl.colors.to_rgba("xkcd:royal blue"))[:3]
    back_palette = (sns.dark_palette(back_color, n_colors=2)[0:1] 
                + [back_color] + sns.light_palette(back_color, n_colors=4)[1:2])
    back_color_samples = np.clip(back_color + 0.25, a_min=0.0, a_max=1.0)
elif back_choice == "colorblind":
    base_palette = [np.asarray(a) for a in sns.color_palette("deep", n_colors=4)]
    # Change the saturation down a little bit for each
    base_hls = [colorsys.rgb_to_hls(*a[:3]) for a in base_palette]
    back_palette = [sns.set_hls_values(base_palette[i], s=base_hls[i][2]-0.1, l=base_hls[i][1]) 
                    for i in range(len(base_palette))]
    back_palette = np.asarray(back_palette)
    back_color = np.clip(back_palette[0] - 0.05, a_min=0.0, a_max=1.0)
    back_color_samples = np.clip(back_palette[0] + 0.1, a_min=0.0, a_max=1.0)
elif back_choice == "teals":
    back_color = np.asarray(mpl.colors.to_rgba("xkcd:greyish blue"))[:3]
    #back_color = np.asarray(mpl.colors.to_rgba("xkcd:dark teal"))
    back_palette = (sns.dark_palette(back_color, n_colors=8)[2:3] 
                + [back_color] + sns.light_palette(back_color, n_colors=6)[2:3])
    back_color_samples = np.clip(back_color + 0.25, a_min=0.0, a_max=1.0)
    #back_palette = [a + (1,) for a in back_palette]
    
new_color = "r"
linestyles = ["-", (0, (5, 1, 2, 1)), "--", ":", "-."]
sns.palplot(back_palette)
plt.show()
plt.close()

In [ ]:
all_back_colors = {
    "back_color": back_color, 
    "back_palette": back_palette, 
    "back_color_samples": back_color_samples
}
colorfile = os.path.join(params_folder, "back_colors.json")
if not os.path.isfile(colorfile):
    with open(colorfile, "w") as f:
        json.dump(all_back_colors, f)

In [ ]:
new_rcParams = {}
new_rcParams["figure.figsize"] = (2.25, 1.75)
new_rcParams["axes.labelsize"] = 7.
new_rcParams["legend.fontsize"] = 6.
new_rcParams["axes.labelpad"] = 1.0
new_rcParams["xtick.labelsize"] = 6.
new_rcParams["ytick.labelsize"] = 6.
new_rcParams["legend.title_fontsize"] = 7.
new_rcParams["legend.handlelength"] = 1.5
new_rcParams["axes.titlesize"] = 7.
new_rcParams["font.size"] = 7.
new_rcParams["figure.dpi"] = 200
new_rcParams["lines.linewidth"] = 1.0
new_rcParams["lines.markeredgewidth"] = 0.5
new_rcParams["axes.linewidth"] = 0.6
new_rcParams['axes.spines.top'] = False
new_rcParams['axes.spines.right'] = False
for tick in ["xtick", "ytick"]:
    new_rcParams[tick + ".major.size"] = 2.5
    new_rcParams[tick + ".minor.size"] = 1.75
    new_rcParams[tick + ".major.width"] = new_rcParams["axes.linewidth"]
    new_rcParams[tick + ".minor.width"] = new_rcParams["axes.linewidth"] - 0.2
    new_rcParams[tick + ".major.pad"] = 2.5
    new_rcParams[tick + ".minor.pad"] = 2.0

# Save these rcParams to disk for all figures
rcfile = os.path.join(params_folder, "olfaction_rcparams.json")
if not os.path.isfile(rcfile):
    with open(rcfile, "w") as f:
        json.dump(new_rcParams, f)

# Load aesthetic parameters
Code for each figure to load a consistent set of aesthetic parameters

In [ ]:
# rcParams
with open(os.path.join(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(os.path.join(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

# Panel A: concentration time series

In [ ]:
# Load turbulent concentration time series
nuser_example = np.load(os.path.join(data_folder, "sample_turbulent_background.npz"))["nuser"]
tser_example = np.load(os.path.join(data_folder, "sample_turbulent_background.npz"))["tser"]
n_odors_example = 3
skp_example = 1
dt_u = 10  # ms
conc_ser_example = nuser_example[:, :n_odors_example, 1]
new_odor_example = nuser_example[:, n_odors_example, 1]

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(3.0, plt.rcParams["figure.figsize"][1]*1.02)
# Add new odor only at the end of the time series, under other odors
t_window = slice(15500 // skp_example, 17500 // skp_example, 1)
t_window_new = slice(2550 // skp_example, 2900 // skp_example, 1)
new_odor_snapshot = new_odor_example[t_window_new]
t_axis = (tser_example[t_window] - tser_example[t_window][0])/1000*dt_u
ax.plot(t_axis[-len(new_odor_snapshot):], new_odor_snapshot,
       color=new_color, lw=plt.rcParams["lines.linewidth"]*1.25, ls="-.")
first_nonzero_idx_new = np.nonzero(new_odor_snapshot)[0][0]
ax.annotate("New odor", xy=(t_axis[-len(new_odor_snapshot)+first_nonzero_idx_new], new_odor_snapshot.max()*1.05), 
            xycoords="data", ha="center", va="bottom", color=new_color, rotation=90)
# Plot background odors on top
for i in range(n_odors_example):
    ax.plot(t_axis, conc_ser_example[t_window, (i+1)%3], label=" ", 
           color=back_palette[i], lw=plt.rcParams["lines.linewidth"]*0.75)

ax.legend(frameon=False, handleheight=3.0, loc="upper left", bbox_to_anchor=(0.0, 1.1))
ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(5))
ax.set_xlabel("Time (s)")
ax.set_ylabel("Concentration (a.u.)", labelpad=0.0)
# Concentration is arbitrary, remove axis
ax.set_yticks([0])
ax.set_ylim((0, ax.get_ylim()[1]))
fig.tight_layout()
if do_save_plots:
       fig.savefig(os.path.join(panels_folder, "sample_turbulent_concentrations_time_series_{}.pdf".format(back_choice)), 
           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel B: background process
Histograms of concentrations, whiff duration and blank duration, overlaid with analytical distributions. 

In [ ]:
ex2 = np.load(os.path.join(data_folder, "sample_turbulent_background.npz"))
# Time units per simulation step, for plotting, in ms
dt_u = 10.0  # ms

In [ ]:
def hist_outline(ax, bins, height, **plot_kwargs):
    plot_hist = np.stack([height, height], axis=1).flatten()
    plot_edges = np.stack([bins[:-1], bins[1:]], axis=1).flatten()
    ax.plot(plot_edges, plot_hist, **plot_kwargs)
    ax.fill_between(plot_edges, min(0.0, height.min()), plot_hist,
                    color=plot_kwargs.get("color"), alpha=0.3)
    return ax

In [ ]:
# Plot the statistics to make sure it's convenient and correct.
fig = plt.figure()
gs = fig.add_gridspec(1, 6)
axes = [fig.add_subplot(gs[:, :2]), fig.add_subplot(gs[:, 2:4]), fig.add_subplot(gs[:, 4:6])]
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*2.0, fig.get_size_inches()[1]*1.05)

# Concentrations
ax = axes[0]
# Histogram
hist_outline(ax, bins=ex2["conc_bins"], height=ex2["conc_pdf"], color=back_color_samples, label="Samples")
# Analytical
conc_axis = np.linspace(ex2["conc_bins"][0], ex2["conc_bins"][-1], 201)
ax.plot(conc_axis,  ex2["conc_analytical_pdf"], color="k", lw=1.0,
        label=r"$p_c(c) \sim \frac{e^{-c/c_0}}{Ac}$")
# Probability of zero concentration
ax.plot(0.0, ex2["conc_prob_zero"], mfc=back_palette[2],  #, label="Blank (c=0)",
        marker="o", ls="none", mec=back_palette[1], ms=4)
ax.plot(0.0, ex2["conc_analytical_prob_zero"], marker="*", color="k", ls="none",
        mec="k", ms=2)  #, label=r"$p_c(c=0) = 1 - \chi$")
# Annotate with arrow since does not fit in legend
ax.annotate("", xy=(0.1, ex2["conc_prob_zero"]*0.5), xytext=(1.0, 2e-3), 
            arrowprops=dict(color=back_color_samples, width=0.3, headwidth=3.0, headlength=3.0), 
            ha="center", va="top")
ax.annotate("Blanks", xy=(1.0, 1e-3), ha="center", va="top")
ax.legend(frameon=False, handletextpad=0.5)
ax.set(xlabel=r"$c$ (a.u.)", yscale="log")
ax.set_ylabel(ylabel=r"$p_c(c)$", labelpad=0.5)
ax.set_title(r"Concentration $c$", va="top", pad=-20)

# Whiffs: just say the distribution is similar but stops at shorter times
pdf_fact = 1.0 / (dt_u/1000)  # to have s^{-1} units
if len(axes) > 2:
    ax = axes[1]
    hist_outline(ax, ex2["t_w_bins"]*dt_u/1000, ex2["t_w_pdf"]*pdf_fact, 
                 color=back_color_samples, label="Samples")
    ax.plot(ex2["t_w_axis"]*dt_u/1000, ex2["t_w_analytical_pdf"]*pdf_fact, color="k", lw=1.0,
            label=r"$p(t_\mathrm{w}) \sim t_\mathrm{w}^{-3/2}$")
    ax.set(xlabel=r"$t_\mathrm{w}$ (s)", yscale="log", xscale="log")
    ax.set_ylabel(ylabel=r"$p_t(t_\mathrm{w})$ (s$^{-1}$)", labelpad=0.5)
    ax.legend(frameon=False, handletextpad=0.5)
    ax.set_title(r"Whiff duration $t_{\mathrm{w}}$", va="top", pad=-20)
    
# Blanks
ax = axes[-1]
hist_outline(ax, ex2["t_b_bins"]*dt_u/1000, ex2["t_b_pdf"]*pdf_fact, 
             color=back_color_samples, label="Samples")
# Analytical
ax.plot(ex2["t_b_axis"]*dt_u/1000, ex2["t_b_analytical_pdf"]*pdf_fact, color="k", lw=1.0,
        label=r"$p(t_\mathrm{b}) \sim t_\mathrm{b}^{-3/2}$")
ax.set(xlabel=r"$t_\mathrm{b}$ (s)", yscale="log", xscale="log")
ax.set_ylabel( ylabel=r"$p_t(t_\mathrm{b})$ (s$^{-1}$)", labelpad=0.5)
ax.legend(frameon=False, loc="upper right", handletextpad=0.5)
ax.set_title(r"Blank duration $t_\mathrm{b}$", va="top", pad=-20)
   
fig.tight_layout(w_pad=0.2)
if do_save_plots:
        fig.savefig(os.path.join(panels_folder, "turbulent_background_statistics_3panels.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel D: curved 2D background manifold in 3D space

In [ ]:
def curved_manifold(x, y, params):
    alpha, beta, z0 = params
    z = -alpha*(x**2 + y**2) + beta*(x+y) + z0
    return z

def manifold_normal_vec(x, y, params):
    """ Normal vector to the manifold at any x, y point is
    the cross product of its directional derivatives along x, y. 
    """
    alpha, beta, z0 = params
    dx = 2*alpha*x - beta
    dy = 2*alpha*y - beta
    dz = 1.0
    v = np.asarray([dx, dy, dz])
    return v / np.sqrt(np.sum(v**2))

In [ ]:
sq2 = np.sqrt(2)
back_components = np.asarray([[0, 0.1, 1.0], [0.1, 0, 1.0]])
back_components = back_components / np.sqrt(np.sum(back_components**2, axis=1, keepdims=True))
# Add a third vector in that linear space, to indicate we typically have more than 2 odors
#back_components_extra = np.concatenate([back_components, 
#        -back_components[0:1]*0.3 + back_components[1:2]], axis=0)
#back_components_extra[2] /= np.sqrt(np.sum(back_components_extra[2]**2))

# Importantly, plot a continuous surface for this curved manifold
curve_params = [0.2, -0.1, 10.0]
surface_x1, surface_x2 = np.meshgrid(np.arange(-1, 1, 0.04), np.arange(-1, 1, 0.04))
surface_z = curved_manifold(surface_x1, surface_x2, curve_params)

# Place sample points on this manifold
# Use log-normal samples: less dire, illustrates low-d manifold better
n_samples = 100
nuser = np.load(os.path.join(data_folder, "sample_lognormal_simulation.npz"))["nuser"]
conc_samples = 10.0**nuser[35:35+n_samples, :2] * 6.0
nu_avg = np.mean(10.0**nuser)

back_vecs_samples = conc_samples.dot(back_components)
origin_vectors = (-0.8, -0.8)
back_vecs_samples[:, :2] += origin_vectors[0]
back_vecs_samples[:, 0] = np.clip(back_vecs_samples[:, 0], surface_x1.min()+0.2, surface_x1.max()-0.2)
back_vecs_samples[:, 1] = np.clip(back_vecs_samples[:, 1], surface_x1.min()+0.2, surface_x1.max()-0.2)
# To offset just a little bit all points, so we can see them
#normal_vec = np.cross(back_components[0], back_components[1])
#back_vecs_samples = back_vecs_samples[conc_samples.max(axis=1) < 4] + 0.05*normal_vec[None, :]

# Curve the z coordinate
back_vecs_samples[:, 2] = curved_manifold(back_vecs_samples[:, 0], back_vecs_samples[:, 1], curve_params) + 0.01

# New odor: above the manifold
xy_new_odor = np.asarray((0.97, 1.2)) + origin_vectors
origin_on_manifold = np.asarray((*origin_vectors, curved_manifold(*origin_vectors, curve_params)))
# Normal at that point
normal_new_odor = 0.8 * manifold_normal_vec(*xy_new_odor, curve_params)
new_odor_on_manifold = np.asarray((*xy_new_odor, curved_manifold(*xy_new_odor, curve_params)))
new_odor_tip = new_odor_on_manifold.copy() + normal_new_odor
new_odor_line_join = np.stack([new_odor_on_manifold, new_odor_tip], axis=0)
vector_new_odor = new_odor_tip - origin_on_manifold

# x_parallel line
new_odor_parallel = origin_on_manifold.reshape(3, 1) + np.arange(0.0, 1.0, 0.02) * (new_odor_on_manifold - origin_on_manifold).reshape(3, 1)
new_odor_parallel[2] = curved_manifold(new_odor_parallel[0], new_odor_parallel[1], curve_params)

In [ ]:
fig = plt.figure()
# Make figure wider, there is always lost whitespace in 3d plots
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*1.2, 
                    plt.rcParams["figure.figsize"][1]*1.2)

ax = fig.add_subplot(projection='3d')
ax.plot_surface(surface_x1, surface_x2, surface_z, color="grey", alpha=0.5)
ax.plot(back_vecs_samples[:, 0], back_vecs_samples[:, 1], back_vecs_samples[:, 2], 
        ls="-", marker="o", alpha=0.8, color=back_color_samples, ms=2, lw=0.5)

# Plot new odor: total, parallel, orthogonal components
ax.plot(*new_odor_line_join.T, ls="--", color=new_color, lw=1.0)
ax.plot(*new_odor_tip, marker="*", mfc=new_color, mec="k", ms=6)
ax.quiver(*origin_on_manifold, *(vector_new_odor*0.96), color=new_color, linewidth=1.0, arrow_length_ratio = 0.15)
ax.plot(new_odor_on_manifold[0], new_odor_on_manifold[1], new_odor_on_manifold[2]+0.01, mfc="k", mec=new_color, ms=6)
ax.plot(*new_odor_parallel, color="k", linewidth=1.0)
for lbl, f in enumerate([ax.set_xlabel, ax.set_ylabel, ax.set_zlabel]):
    f(r"$s_{}$".format(lbl+1), labelpad=-15.0)
ax.zaxis.set_rotate_label(False)
for f in [ax.set_xticks, ax.set_yticks, ax.set_zticks]:
    f([])
for f in [ax.set_xticklabels, ax.set_yticklabels, ax.set_zticklabels]:
    f([], pad=0.1)
#for lbl, axis in enumerate([ax.xaxis, ax.yaxis, ax.zaxis]):
    #axis.set_pane_color((1.0, 1.0, 1.0, 1.0))

# Annotate background
ax.text(surface_x1.max(), 0.0, surface_z.min()*0.99, "Background", (0.25, 1, -0.00), ha="center", 
        va="center", color=back_color_samples)
# Annotate new odor
ax.text(*(new_odor_tip*1.01), "New odor", None, color=new_color, ha="center", va="bottom")
label_pos = np.asarray((new_odor_on_manifold[0], new_odor_on_manifold[1]+0.1, new_odor_on_manifold[2]))
ax.text(*(label_pos + 0.5*normal_new_odor) , r"$\mathbf{s}_{\mathrm{new},\perp}$", None, color=new_color, ha="left", va="bottom")
label_pos = origin_on_manifold + 0.3 * vector_new_odor
label_pos[2] += 0.1
ax.text(*label_pos , r"$\mathbf{s}_\mathrm{new}$", None, color=new_color, ha="left", va="bottom")
label_pos = new_odor_parallel[:, 7*new_odor_parallel.shape[1]//8]
ax.text(*label_pos , r"$\mathbf{s}_{\mathrm{new},//}$", None, color="k", ha="right", va="top")

# Plot limits
ax.set_zlim([surface_z.min()*0.95, surface_z.max()*1.05])
ax.view_init(azim=25, elev=20)
ax.set_aspect("equal")
fig.tight_layout()
# Need to adjust the tightbox to remove whitespace above and below manually. 
tightbox = fig.get_tightbbox()
tightbox._bbox.y0 = tightbox._bbox.y0*1.6
tightbox._bbox.y1 = tightbox._bbox.y1 - 0.8*tightbox._bbox.y0
tightbox._bbox.x0 = tightbox._bbox.x0 * 0.7
if do_save_plots:
    fig.savefig(os.path.join(panels_folder, "fig1_manifold_cartoon_curved_{}.pdf".format(back_choice)), 
            bbox_inches=tightbox, transparent=True)
plt.show()
plt.close()

# Panel E: predictive filtering vs manifold learning transition

In [ ]:
loss_maps = dict(np.load("../../results/for_plots/manifold_learning_heatmap_data.npz"))

In [ ]:
def phase_boundary_v_p(tau_range):
    n_r = 1.0 / (np.exp(2.0/tau_range) - 1.0)
    return n_r

In [ ]:
tau_grid, n_r_grid = np.meshgrid(loss_maps["tau_range"], loss_maps["n_s_range"], indexing="xy")
# Compute the phase
loss_ratio_v = np.log10(loss_maps["v"] - loss_maps["vp"])
loss_ratio_p = np.log10(loss_maps["p"] - loss_maps["vp"])
phase = loss_ratio_v - loss_ratio_p
# Normalization for the heatmap
max_ampli = np.amax(np.abs(phase))
fig, ax = plt.subplots()
xtent = (tau_grid.min(), tau_grid.max(), n_r_grid.min(), n_r_grid.max())
im = ax.imshow(phase, cmap="RdGy_r", vmin=-max_ampli, vmax=max_ampli,  # or "bwr" map if I prefer
    extent=xtent, origin="lower", 
)
# Analytical boundary
nr_boundary = phase_boundary_v_p(loss_maps["tau_range"])
ax.plot(loss_maps["tau_range"], nr_boundary, ls="--", color="k")
ax.set_xlim(xtent[0], xtent[1])
ax.set_ylim(xtent[2], xtent[3])
ax.set_xlabel(r"Autocorrelation time, $\tau$ (a.u.)")
ax.set_ylabel(r"Scaled dimension, $\tilde{N}_\mathrm{S}$")
# $\tilde{N_\mathrm{S}} = \frac{\sigma^2}{\sigma_x^2}$

cbar = fig.colorbar(im, ax=ax)
cbar.set_label(r"Regime: $\Phi = \log_{10}\left(\frac{\mathcal{L}_v - \mathcal{L}_{v, P}}"
        + r"{\mathcal{L}_P - \mathcal{L}_{v, P}}\right)$", labelpad=4.0)
cbar.ax.set_yticks([-0.7*max_ampli, 0, 0.7*max_ampli])
cbar.ax.tick_params(axis="y", length=0.0, pad=1.0)
cbar.ax.set_yticklabels([r"$-$", r"$0$", r"$+$"])
tau_mid = loss_maps["tau_range"][loss_maps["tau_range"].shape[0] // 3]
ax.annotate(r"$\tau \sim 2\left(\tilde{N}_\mathrm{S} + 1\right)$", 
            (tau_mid, tau_mid/2.0 + 25.0), rotation=25, fontsize=6)
ax.annotate("Manifold learning", #r"$W$ dominates", 
    (loss_maps["tau_range"][loss_maps["tau_range"].shape[0] // 8],
      loss_maps["n_s_range"][loss_maps["n_s_range"].shape[0]//16 * 15]), 
    va="top"
)
ax.annotate("Predictive\nfiltering", #r"$v$ dominates", 
    (loss_maps["tau_range"][-10], loss_maps["n_s_range"][loss_maps["n_s_range"].shape[0]//20]), 
    ha="right", va="bottom")
fig.tight_layout()

# Put an ellipse with "Olfaction" in it
olf_center = (loss_maps["tau_range"][loss_maps["tau_range"].shape[0] // 16 * 6],
      loss_maps["n_s_range"][loss_maps["n_s_range"].shape[0]//16 * 10])
ell = mpl.patches.Ellipse(olf_center, loss_maps["tau_range"][-1]*0.55, loss_maps["n_s_range"][-1]*0.3, 
    edgecolor='k', linestyle="--", linewidth=0.5, facecolor=(1.0, 1.0, 1.0, 0.0)
)
ax.add_artist(ell)
ell.set_clip_box(ax.bbox)
ax.annotate("Olfaction", olf_center, va="center", ha="center", fontweight="bold")
fig.tight_layout()
if do_save_plots:
    fig.savefig("panels/fig1_loss_manifold_learning_heatmap.pdf", transparent=True, bbox_inches="tight")
plt.show()
plt.close()